# Script to remove error values

In [24]:
import pandas as pd
import datetime

df = pd.read_excel(r'/Users/jorgevieira/Desktop/Universidade/FEUP/YEAR 1/1st SEM/FCED/STATS/FCED-STATS/Lista Incendios_2015.xlsx')
sample = df.sample(frac = 0.3)
print(len(sample.index))


for index, row in sample.iterrows():

    if (isinstance(row.DataAlerta, str) and isinstance(row.Data1Intervencao, str) and isinstance(row.DataExtincao, str) and isinstance(row.x, int) and isinstance(row.TipoCausa, str)):
        #print(index)
        d1 = datetime.datetime.strptime(row.DataAlerta[:-4], '%Y-%m-%d %H:%M:%S')
        d2 = datetime.datetime.strptime(row.Data1Intervencao[:-4], '%Y-%m-%d %H:%M:%S')
        d3 = datetime.datetime.strptime(row.DataExtincao[:-4], '%Y-%m-%d %H:%M:%S')
        #print(d1)
        #print(d2)
        #print('-------------')
    else:
        #print('NAO E STRING')
        sample.drop(index, inplace=True, axis=0)
        continue
        #print('DROPPED' + str(index))
    
    if(row.x <= 0):
        sample.drop(index, inplace=True, axis=0)
        continue

    if (d2 > d3 or d1 > d3):
        sample.drop(index, inplace=True, axis=0)
        continue
    
    elif (d2 == d3 or d1 == d3):

        if (isinstance(row.HoraAlerta, datetime.time) and isinstance(row.Hora1Intervencao, datetime.time) and isinstance(row.HoraExtincao, datetime.time)): 
            if (d2 == d3 and row.HoraExtincao < row.Hora1Intervencao):
                #print('HORAS NAO BATEM')
                sample.drop(index, inplace=True, axis=0)
                continue
                #print('DROPPED' + str(index))
            if (d1 == d3 and row.HoraExtincao < row.HoraAlerta):
                #print('HORAS NAO BATEM')
                sample.drop(index, inplace=True, axis=0)
                continue
                #print('DROPPED' + str(index))
        else:
            sample.drop(index, inplace=True, axis=0)
            continue

print(len(sample.index))
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#print(sample.head(n=5))

Ano                                      int64
Codigo SGIF                             object
Codigo_ANPC                              int64
Tipo                                    object
Distrito                                object
Concelho                                object
Freguesia                               object
Local                                   object
INE                                      int64
x                                        int64
y                                        int64
lat                                     object
lon                                     object
DataAlerta                              object
HoraAlerta                              object
DataExtincao                            object
HoraExtincao                            object
Data1Intervencao                        object
Hora1Intervencao                        object
FonteAlerta                            float64
NUT                                     object
AA_Povoamento

# Step 3 : Explore the data

In [25]:
from datetime import datetime, date
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing

regiao = []
for line in sample.x:
    if(line > 212000):
        regiao.append('Interior')
    else:
        regiao.append('Litoral')
        

datasAlerta = list(map(lambda x: datetime.strptime(x[:10],'%Y-%m-%d').date(),sample.DataAlerta.values))
datasInter = list(map(lambda x: datetime.strptime(x[:10],'%Y-%m-%d').date(),sample.Data1Intervencao.values))
datasExtincao = list(map(lambda x: datetime.strptime(x[:10],'%Y-%m-%d').date(),sample.DataExtincao.values))

dataHoraAlerta = []
for line1, line2 in zip(sample["HoraAlerta"].values,datasAlerta):
        dataHoraAlerta.append(datetime.combine(line2,line1))  
        
dataHoraInter = []
for line1, line2 in zip(sample["Hora1Intervencao"].values,datasInter):
        dataHoraInter.append(datetime.combine(line2,line1))
        
dataHoraExtincao = []
for line1, line2 in zip(sample["HoraExtincao"].values,datasExtincao):
        dataHoraExtincao.append(datetime.combine(line2,line1))

def timediff(l1, l2):
    res = []
    for line1, line2 in zip(l1,l2):
        res.append(line1-line2)
    return res

tempoExtincao = timediff(dataHoraExtincao,dataHoraInter)
duracaoFogo = timediff(dataHoraExtincao,dataHoraAlerta)

dict = {'TempoExtincao' : tempoExtincao, 'DuracaoFogo' : duracaoFogo, 'AreaArdida' : sample['AA_Total (pov+mato+agric) (ha)'], 'Regiao' : regiao, 'TipoCausa' : sample.TipoCausa}

newdf = pd.DataFrame(dict)

In [26]:
newdf.apply(lambda x: sum(x.isnull()), axis=0)

TempoExtincao    0
DuracaoFogo      0
AreaArdida       0
Regiao           0
TipoCausa        0
dtype: int64

In [27]:
newdf.describe(include='all')

,TempoExtincao,DuracaoFogo,AreaArdida,Regiao,TipoCausa
count,4670,4670,4670.000000,4670,4670
unique,NaN,NaN,NaN,2,5
top,NaN,NaN,NaN,Litoral,Negligente
freq,NaN,NaN,NaN,3322,1700
mean,0 days 01:55:56.955032119,0 days 02:09:41.794432548,3.217814,NaN,NaN
std,0 days 05:19:11.161762454,0 days 05:20:11.145305637,37.835976,NaN,NaN
min,0 days 00:00:00,0 days 00:02:00,0.000000,NaN,NaN
25%,0 days 00:47:00,0 days 00:59:00,0.010000,NaN,NaN
50%,0 days 01:18:00,0 days 01:31:00,0.070450,NaN,NaN
75%,0 days 02:10:00,0 days 02:27:00,0.500000,NaN,NaN
